In [13]:
import csv
import datetime

f = open("guns.csv", "r")
csvreader = csv.reader(f)
data = list(csvreader)

print(data[0:5])



[['', 'year', 'month', 'intent', 'police', 'sex', 'age', 'race', 'hispanic', 'place', 'education'], ['1', '2012', '01', 'Suicide', '0', 'M', '34', 'Asian/Pacific Islander', '100', 'Home', '4'], ['2', '2012', '01', 'Suicide', '0', 'F', '21', 'White', '100', 'Street', '3'], ['3', '2012', '01', 'Suicide', '0', 'M', '60', 'White', '100', 'Other specified', '4'], ['4', '2012', '02', 'Suicide', '0', 'M', '64', 'White', '100', 'Home', '4']]


In [2]:
headers = data[0]
print(headers)

data = data[1:len(data)]
print(data[2:5])

['', 'year', 'month', 'intent', 'police', 'sex', 'age', 'race', 'hispanic', 'place', 'education']
[['3', '2012', '01', 'Suicide', '0', 'M', '60', 'White', '100', 'Other specified', '4'], ['4', '2012', '02', 'Suicide', '0', 'M', '64', 'White', '100', 'Home', '4'], ['5', '2012', '02', 'Suicide', '0', 'M', '31', 'White', '100', 'Other specified', '2']]


In [3]:
year_counts = {}
count = 0;
for d in data:
    if d[1] in year_counts:
        year_counts[d[1]] += 1
    else:
        year_counts[d[1]] = 1
print(year_counts)

{'2013': 33636, '2014': 33599, '2012': 33563}


##### Guns death did not change much year by year

Let'see how it change monthly

In [4]:
from datetime import datetime

date_counts = {}

for d in data:
    date = datetime(year = int(d[1]), month = int(d[2]), day = 1)
    
    if date in date_counts:
        date_counts[date] += 1
    else:
        date_counts[date] = 1  


In [5]:
inverse = [(value, key) for key, value in date_counts.items()]
maxim = max(inverse)[1]
print(maxim, date_counts[maxim])

minim = min(inverse)[1]
print(minim, date_counts[minim])

2013-07-01 00:00:00 3079
2012-02-01 00:00:00 2357


As we can observe, between the max and min value in date_counts we have 600-650 units which can be representative.

### Exploring Gun Deaths by Race and Sex

In [6]:
sex_counts = {}
race_counts = {}
for d in data:
    if d[5] in sex_counts:
        sex_counts[d[5]] += 1
    else:
        sex_counts[d[5]] = 1
    if d[7] in race_counts:
        race_counts[d[7]] += 1
    else:
        race_counts[d[7]] = 1

print(sex_counts)
print(race_counts)

{'M': 86349, 'F': 14449}
{'Black': 23296, 'White': 66237, 'Asian/Pacific Islander': 1326, 'Native American/Native Alaskan': 917, 'Hispanic': 9022}


#### Results:
    As we can observe: 
            - Male race is who died more by guns.
            - White is the race who died more by guns
            

In [7]:
#lets open a new file census.csv on we can find the ppl censed.
census_file = open("census.csv", "r")
censusreader = csv.reader(census_file)
census = list(censusreader)

print(census)

[['Id', 'Year', 'Id', 'Sex', 'Id', 'Hispanic Origin', 'Id', 'Id2', 'Geography', 'Total', 'Race Alone - White', 'Race Alone - Hispanic', 'Race Alone - Black or African American', 'Race Alone - American Indian and Alaska Native', 'Race Alone - Asian', 'Race Alone - Native Hawaiian and Other Pacific Islander', 'Two or More Races'], ['cen42010', 'April 1, 2010 Census', 'totsex', 'Both Sexes', 'tothisp', 'Total', '0100000US', '', 'United States', '308745538', '197318956', '44618105', '40250635', '3739506', '15159516', '674625', '6984195']]


### Computing rates of guns deaths per race per 100000

In [8]:
#Criminal rates always come with a rate *100000
mapping = {}
    #lets get the index of the items we need
    #For Asian/Pacific Islander

mapping['Asian/Pacific Islander'] = census[1][census[0].index('Race Alone - Asian')] + census[1][census[0].index('Race Alone - Native Hawaiian and Other Pacific Islander')]

    #For black and the rest we have to look for just one index
mapping['Black'] = census[1][census[0].index('Race Alone - Black or African American')]
mapping['Hispanic'] = census[1][census[0].index('Race Alone - Hispanic')]
mapping['Native American/Native Alaskan'] = census[1][census[0].index('Race Alone - American Indian and Alaska Native')]
mapping['White'] = census[1][census[0].index('Race Alone - White')]

print(mapping)



{'Black': '40250635', 'White': '197318956', 'Native American/Native Alaskan': '3739506', 'Asian/Pacific Islander': '15159516674625', 'Hispanic': '44618105'}


##### Now we have all we need to get the rate *100000 = (race_counts/mapping ) *100K

In [9]:
race_per_hundredk = {}
for row in race_counts:
    race_per_hundredk[row] = (float(race_counts[row])/float(mapping[row]))*100000
print(race_per_hundredk)   
    

{'Black': 57.8773477735196, 'White': 33.56849303419181, 'Asian/Pacific Islander': 8.746980714890115e-06, 'Native American/Native Alaskan': 24.521955573811088, 'Hispanic': 20.220491210910907}


### Filtering by Intent

In [10]:
intents = []
for d in data:
    if d[headers.index('intent')] == 'Homicide':
        intents.append(d)

#lets extract the race
races = []
races_count_homic = {}
for i in intents:
    races.append(i[7])
    if i[7] in races_count_homic:
        races_count_homic[i[7]] += 1
    else:
        races_count_homic[i[7]] = 1
    
    
print(races_count_homic)  

{'Black': 19510, 'White': 9147, 'Asian/Pacific Islander': 559, 'Native American/Native Alaskan': 326, 'Hispanic': 5634}


##### Let's extract the rate of homicides per race *100000

In [11]:
homicide_race_per_hundredk = {}

for c in races_count_homic:
    homicide_race_per_hundredk[c] = (races_count_homic[c]/float(mapping[c]))*100000

print(homicide_race_per_hundredk)

{'Black': 48.471284987180944, 'White': 4.6356417981453335, 'Asian/Pacific Islander': 3.687452654316421e-06, 'Native American/Native Alaskan': 8.717729026240365, 'Hispanic': 12.627161104219914}


### Results

###### Black race is who more comitted homicide per 100K
###### Asian/Pacific islanders who less homic comitted per 100K

## Generalizating some research
#### Here I am going to use functions that extract the result we are looking for in order to get the total of guns death for each intent for instance or also for gender.


In [19]:
def deaths_by_intent_100K(intent):
    
    intents = []
    for d in data:
        if d[headers.index('intent')] == intent:
            intents.append(d)

    #lets extract the race
    races = []
    races_count_intent = {}
    for i in intents:
        races.append(i[7])
        if i[7] in races_count_intent:
            races_count_intent[i[7]] += 1
        else:
            races_count_intent[i[7]] = 1
    
    
    intent_race_per_hundredk = {}

    for c in races_count_intent:
        intent_race_per_hundredk[c] = (races_count_intent[c]/float(mapping[c]))*100000

    return(intent_race_per_hundredk)

death_accidental = deaths_by_intent_100K('Accidental')
print(death_accidental)

{'Black': 0.814893976206835, 'White': 0.5736904466492313, 'Native American/Native Alaskan': 0.5883130017708221, 'Asian/Pacific Islander': 7.915819651484268e-08, 'Hispanic': 0.3249801846133985}
